## MVP Engenharia de dados
Projeto desenvolvido para a disciplina de Engenharia de dados. Para desenvolvimento do projeto, será usada base de dados com informações sobre tempo de tela entre crianças e adolescentes obtida no repositório kaggle: https://www.kaggle.com/datasets/ak0212/average-daily-screen-time-for-children

Objetivos: com o sucesso da série "Adolescente" lançada recentemente, observou-se aumento de discussões sobre a exposição de crianças e adolescentes a conteúdo online. Assim, as analises desse projeto visão entender qual o tempo médio de exposição a telas entre diferentes faixas etárias, qual o gênero com maior tempo de exposição a tela, qual o tipo de acesso mais frequente entre os usuários.


Para alcançar os objetivos, serão usados os recursos da Azure, Databricks e Power Bi, para carregar, tratar e visualizar os dados. Também serão realizadas analises iniciais para avaliação da qualidade das informações. Por fim, será realizado estudo para alcançar os objetivos apresentados inicialmente.


# 1. Recurso de cloud: Azure

  Para realizar o processo de armazenamento em nuvem, foi escolhida a ferramenta Azure, usada na versão de teste gratuito. A escolha foi feita para simular um ambiente de trabalho com uso de múltiplo recursos operacionais e para ampliar o conhecimento sobre novas ferramentas.

  O ambiente foi configurado para ter o grupo de recursos contendo conta de armazenamento, Data Factory e conexão com serviço do Azure Databricks.

![ambiente_azure_tela_inicial.png](./ambiente_azure_tela_inicial.png "ambiente_azure_tela_inicial.png")

  Na ferramenta da Azure, foi criado o container "mvp" e a pasta "raw" que serão usadas para armazenamento de arquivo em nuvem.
 ![container_azure.png](./container_azure.png "container_azure.png")

  Para simular um ambiente de produção, foi escolhido baixar o arquivo disponibilizado no site https://www.kaggle.com/datasets/ak0212/average-daily-screen-time-for-children, e carregar no github para que fosse possível criar um conjunto de dados no Azure Data Factory que realiza a leitura de arquivos csv com fonte Http.
  ![github.png](./github.png "github.png")
  No ambiente do data factory, foram criados dois serviços vinculados, para leitura de arquivo com fonte http e para armazenamento em blob.
  ![serviços_vinculados.png](./serviços_vinculados.png "serviços_vinculados.png")
  Para construção do pipeline, foram criados dois conjuntos de dados. O conjunto "get_data" para ler o arquivo no github: ![get_data.png](./get_data.png "get_data.png")

  E o conjunto "upload_data", para armazenamento em blob:
  ![load_data.png](./load_data.png "load_data.png")

 Assim, foi possível estabelecer o pipeline que faria a carga do csv, usando os parâmetros do conjunto "get_data" como origem e os parâmetros do "upload_data" como coleta. Ao depurar o pipeline, o arquivo csv foi corretamente carregado na conta de armazenamento:
 ![pipeline.png](./pipeline.png "pipeline.png")

 ![screen_time_container.png](./screen_time_container.png "screen_time_container.png")
  
  Uma vez que o arquivo foi armazenado em nuvem, foi criado notebook no Databricks para dar continuidade ao projeto.


# 2. Lendo arquivos da nuvem
Conectando Databricks com armazenamento Azure e observando se o dataframe criado corretamente.

In [0]:

# Configurar a chave de acesso da conta de armazenamento
spark.conf.set("fs.azure.account.key.mvpengdado.dfs.core.windows.net", "UiP8dafkrcGZd3/THhfKIVRv7WMEYn1NFulhXEh7pvZK+EAHO/uDMsy2xurLyHQLB17fF8s8M89M+ASt1x2Efg==")

# Definir o caminho do arquivo CSV
file_path = "abfss://mvp@mvpengdado.dfs.core.windows.net/raw/Leticia-Moraes-Souza/MVP_ENGENHARIA_DE_DADOS/refs/heads/main/screen_time.csv"

# Carregar o arquivo CSV no DataFrame
df = spark.read.option("header", "true").csv(file_path)

# Mostrar primeiras linhas do data frame
display(df.limit(5))


Age,Gender,Screen Time Type,Day Type,Average Screen Time (hours),Sample Size
5,Male,Educational,Weekday,0.44,500
5,Male,Recreational,Weekday,1.11,500
5,Male,Total,Weekday,1.55,500
5,Male,Educational,Weekend,0.5,500
5,Male,Recreational,Weekend,1.44,500


O dataframe foi criado corretamente, como mostrado pela exibição das 5 primeiras linhas.

# 3. Analises dos dados carregados

Inicialmente, será analisado o formato dos dados de cada coluna e algumas analises descritivas básicas.

In [0]:
# Importando funções do pyspark
import pyspark.sql.functions as F

In [0]:
# Mostrando o schema dos dados
df.printSchema()

root
 |-- Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Screen Time Type: string (nullable = true)
 |-- Day Type: string (nullable = true)
 |-- Average Screen Time (hours): string (nullable = true)
 |-- Sample Size: string (nullable = true)



Como visto, o conjunto de dados é composto por 6 colunas.

A coluna Age, contém informações de idade, a coluna Gender especifica o gênero, a coluna Sreen Time Type contém os tipos de uso das telas, a coluna Day Type mostra se o uso foi durante o fim de semana ou durante a semana, a coluna Average screen time (hours) contém dados de tempo médio de uso de tela, e a coluna Sample Size contém dados sobre o tamanho da amostra observada.

Inicialmente, as colunas serão renomeadas para que fiquem com cabeçalhos padronizados.

In [0]:
# Renomar colunas
df = df.withColumnRenamed("Screen Time Type", "Screen_Time_Type")
df = df.withColumnRenamed("Average Screen Time (hours)", "Average_Screen_Time_hours")
df = df.withColumnRenamed("Day Type", "Day_Type")
df = df.withColumnRenamed("Sample Size", "Sample_Size")

# Mostrar primeiras linhas do data frame
display(df.limit(5))

Age,Gender,Screen_Time_Type,Day_Type,Average_Screen_Time_hours,Sample_Size
5,Male,Educational,Weekday,0.44,500
5,Male,Recreational,Weekday,1.11,500
5,Male,Total,Weekday,1.55,500
5,Male,Educational,Weekend,0.5,500
5,Male,Recreational,Weekend,1.44,500


Ao observar o schema de dados, também foi visto que as 6 colunas estão em formato string. Então, será ajustado para que as colunas com dados numéricos sejam corretamente armazenadas.

In [0]:
# Mudar tipo de dados das colunas numéricas
df = df.withColumn("Age", df["Age"].cast("int"));
df = df.withColumn("Sample_Size", df["Sample_Size"].cast("int"));
df = df.withColumn("Average_Screen_Time_hours", df["Average_Screen_Time_hours"].cast("float"))

# Conferir schema
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Screen_Time_Type: string (nullable = true)
 |-- Day_Type: string (nullable = true)
 |-- Average_Screen_Time_hours: float (nullable = true)
 |-- Sample_Size: integer (nullable = true)



Agora que as colunas estão com tipos corretos, serão obervadas algumas analises descritivas básicas.

In [0]:
df.describe().show()

+-------+------------------+--------------------+----------------+--------+-------------------------+-----------------+
|summary|               Age|              Gender|Screen_Time_Type|Day_Type|Average_Screen_Time_hours|      Sample_Size|
+-------+------------------+--------------------+----------------+--------+-------------------------+-----------------+
|  count|               198|                 198|             198|     198|                      198|              198|
|   mean|              10.0|                NULL|            NULL|    NULL|        2.993030306064721|            400.0|
| stddev|3.1702935859821872|                NULL|            NULL|    NULL|       1.9228215025960735|63.40587171964375|
|    min|                 5|              Female|     Educational| Weekday|                     0.44|              300|
|    max|                15|Other/Prefer not ...|           Total| Weekend|                     8.19|              500|
+-------+------------------+------------

Pode-se ver que há 198 observações para esse conjunto de dados. 
A coluna Age tem valore de 5 a 15, com média de 10 anos.
A coluna Average_Screen_Time_hours tem valores que vão de aproximadamente 26 minutos a 8 horas.
A coluna Sample_Size tem valores que vão de 300 a 500, com média de 400 observações.

Agora será analisado se as colunas contem valores nulos e posteriormente serão observados todos os valores contidos em cada coluna para analisar se é necessário algum tipo de tratamento para retirada de dados inválidos.
 

In [0]:
# Importar função count
from pyspark.sql.functions import count, when, isnan, col

# Mostrar valores nulos
df.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in df.columns]).show()

+---+------+----------------+--------+-------------------------+-----------+
|Age|Gender|Screen_Time_Type|Day_Type|Average_Screen_Time_hours|Sample_Size|
+---+------+----------------+--------+-------------------------+-----------+
|  0|     0|               0|       0|                        0|          0|
+---+------+----------------+--------+-------------------------+-----------+



Não há valores nulos nas colunas, serão analisados se há valores fora do esperado para as informações de cada coluna.

In [0]:
# Mostrando valores da coluna Age
dropDisDF = df.dropDuplicates(["Age"]).select("Age")
dropDisDF.show(truncate=False)

+---+
|Age|
+---+
|12 |
|13 |
|6  |
|5  |
|15 |
|9  |
|8  |
|7  |
|10 |
|11 |
|14 |
+---+



In [0]:
# Mostrando valores da coluna Gender
dropDisDF = df.dropDuplicates(["Gender"]).select("Gender")
dropDisDF.show(truncate=False)

+-----------------------+
|Gender                 |
+-----------------------+
|Other/Prefer not to say|
|Female                 |
|Male                   |
+-----------------------+



In [0]:
# Mostrando valores da coluna Screen_Time_Type
dropDisDF = df.dropDuplicates(["Screen_Time_Type"]).select("Screen_Time_Type")
dropDisDF.show(truncate=False)

+----------------+
|Screen_Time_Type|
+----------------+
|Total           |
|Educational     |
|Recreational    |
+----------------+



In [0]:
# Mostrando valores da coluna Day_Type
dropDisDF = df.dropDuplicates(["Day_Type"]).select("Day_Type")
dropDisDF.show(truncate=False)

+--------+
|Day_Type|
+--------+
|Weekday |
|Weekend |
+--------+



In [0]:
# Mostrando valores da coluna Average_Screen_Time_hours
dropDisDF = df.dropDuplicates(["Average_Screen_Time_hours"]).select("Average_Screen_Time_hours")
dropDisDF.show(truncate=False)

+-------------------------+
|Average_Screen_Time_hours|
+-------------------------+
|1.63                     |
|5.44                     |
|3.22                     |
|0.73                     |
|5.89                     |
|3.81                     |
|5.36                     |
|1.47                     |
|1.08                     |
|4.95                     |
|5.0                      |
|1.73                     |
|5.23                     |
|2.02                     |
|1.36                     |
|6.93                     |
|4.19                     |
|5.82                     |
|5.5                      |
|2.5                      |
+-------------------------+
only showing top 20 rows



In [0]:
# Mostrando valores da coluna Sample_Size
dropDisDF = df.dropDuplicates(["Sample_Size"]).select("Sample_Size")
dropDisDF.show(truncate=False)

+-----------+
|Sample_Size|
+-----------+
|300        |
|460        |
|360        |
|440        |
|500        |
|340        |
|380        |
|400        |
|320        |
|480        |
|420        |
+-----------+



Aparentemente, não há nenhum valor que deva ser substituído. Para a visualização da coluna Average_Screen_Time_Hours foram mostrados apenas os primeiros 20 valores, mas considerando as informações de analise descritiva, onde o valor mínimo era 0.44 e o valor máximo 8.26, pode-se considerar que ela se encontra dentro de um range esperado.

# 4. Extraindo informações dos dados
 
 Serão feitas as analises para atender os objetivos do projeto.

 Sendo eles: 
 
-  entender qual o tempo médio de exposição a telas entre diferentes faixas etárias;

-  qual o gênero com maior tempo de exposição a tela; 
 
-  qual o tipo de acesso mais frequente entre os usuários.

Primeiramente, vamos analisar se a distribuição de amostras tem uma disparidade muito grande, o que poderia enviesar os resultados observados.

In [0]:
# importando função pyspark
from pyspark.sql.functions import desc

# Mostrando a quantidade de amostras
dfamostra = df.groupby('Age', 'Screen_Time_Type', 'Gender', 'Day_Type').agg({'Sample_Size': 'sum'})
dfamostra = dfamostra.orderBy(desc('Age'))
display(dfamostra)

Age,Screen_Time_Type,Gender,Day_Type,sum(Sample_Size)
15,Educational,Male,Weekday,300
15,Recreational,Female,Weekend,300
15,Recreational,Male,Weekday,300
15,Recreational,Other/Prefer not to say,Weekday,300
15,Educational,Female,Weekend,300
15,Total,Female,Weekend,300
15,Recreational,Female,Weekday,300
15,Total,Male,Weekday,300
15,Total,Other/Prefer not to say,Weekend,300
15,Total,Female,Weekday,300


A quantidade de amostras parece ser bem distribuída entre as diferentes variáveis categóricas, portanto, não sendo considerado um possível viés por predomínio de observações de apenas um tipo.

## 4.1 Objetivo 1: qual o tempo de exposição a telas entre diferentes faixas etárias

In [0]:
# Mostar a média de tempo por idade

df_medias = df.where(df["Screen_Time_Type"] == "Total").groupBy('Age').mean('Average_Screen_Time_hours')
df_medias = df_medias.orderBy(desc("avg(Average_Screen_Time_hours)"))
display(df_medias)

Age,avg(Average_Screen_Time_hours)
15,7.291666666666667
14,6.706666628519694
13,6.176666657129924
12,5.571666717529297
11,5.063333352406819
10,4.506666660308838
9,3.915000081062317
8,3.3916666507720947
7,2.8166666825612388
6,2.2350000143051147


Aqui, podemos ver que o tempo médio de tela é maior quanto maior for a idade do usuário, com a maior média sendo observada para adolescentes de 15 anos.
Uma hipótese para explicar esse fato poderia ser associada ao aumento das demandas escolares que exigem maior tempo para pesquisas online. Será feita uma  analise adicional para observar qual tempo médio para os diferentes tipos de uso por cada faixa etária.

In [0]:
# Mostar a média de tempo por idade e tipo de acesso

df_medias = df.where(df["Screen_Time_Type"] == "Educational").groupBy('Age','Screen_Time_Type').mean('Average_Screen_Time_hours')
df_medias = df_medias.orderBy(desc("avg(Average_Screen_Time_hours)"))
display(df_medias)

Age,Screen_Time_Type,avg(Average_Screen_Time_hours)
15,Educational,2.0266666809717813
14,Educational,1.809999982515971
13,Educational,1.5149999856948853
12,Educational,1.5033333500226338
11,Educational,1.418333351612091
10,Educational,1.2550000150998433
9,Educational,1.0266666809717815
8,Educational,0.9916666746139526
7,Educational,0.8100000123182932
6,Educational,0.6583333214124044


De fato, o tempo de tela para uso educacional aumenta com o aumento da idade, porém os valores observados estão consideravelmente abaixo da média de tempo total. 
Então, também é possível concluir que o uso recreativo é o principal responsável pelo aumento de tempo online. Será analisado esse dado para ver se é possível comprovar essa segunda hipótese.

In [0]:
# Mostrar tempo de tela recreativo

df_medias = df.where(df["Screen_Time_Type"] == "Recreational").groupBy('Age','Screen_Time_Type').mean('Average_Screen_Time_hours')
df_medias = df_medias.orderBy(desc("avg(Average_Screen_Time_hours)"))
display(df_medias)

Age,Screen_Time_Type,avg(Average_Screen_Time_hours)
15,Recreational,5.264999945958455
14,Recreational,4.895000060399373
13,Recreational,4.659999926884969
12,Recreational,4.069999933242798
11,Recreational,3.6416666905085244
10,Recreational,3.2500000397364297
9,Recreational,2.886666695276896
8,Recreational,2.3983333110809326
7,Recreational,2.0083333452542624
6,Recreational,1.5766666531562805


Pode-se concluir que de fato, o uso recreacional é superior ao educacional.
E que o tempo online e a idade são diretamente proporcionais, ou seja, uma variação em uma medida provoca uma variação no mesmo sentido na outra.

## 4.2 Objetivo 2: qual o gênero com maior tempo de exposição a tela

In [0]:
# Mostar a média de tempo por gênero

df_medias = df.where(df["Screen_Time_Type"] == "Total").groupBy('Gender').mean('Average_Screen_Time_hours')
df_medias = df_medias.orderBy(desc("avg(Average_Screen_Time_hours)"))
display(df_medias)

Gender,avg(Average_Screen_Time_hours)
Male,4.605454580350355
Other/Prefer not to say,4.4904545545578
Female,4.373636348681017


Podemos observar que o gênero masculino tem maior tempo médio de uso de telas. Vamos observar também qual o tipo de uso predominante para cada gênero.

In [0]:
# Mostrar tempo de tela por gênero e tipo de acesso

df_medias = df.where(df["Screen_Time_Type"] != "Total").groupBy('Gender','Screen_Time_Type').mean('Average_Screen_Time_hours')
df_medias = df_medias.orderBy(desc("avg(Average_Screen_Time_hours)"))
display(df_medias)

Gender,Screen_Time_Type,avg(Average_Screen_Time_hours)
Male,Recreational,3.377727280963551
Other/Prefer not to say,Recreational,3.3095454302701084
Female,Recreational,3.0995454544370826
Female,Educational,1.2731818177483298
Male,Educational,1.2268181849609723
Other/Prefer not to say,Educational,1.1809091026132756


Observamos que quando o uso de telas é para fins recreativos, os homens tendem a passar mais tempo online. Porém o as mulheres usam mais esses recursos quando tem o objetivo educacional.

## 4.3 Objetivo 3: Tipo de acesso mais frequente entre usuários

Para essa analise, vamos considerar que como crianças pessoas que estão na faixa etária entre 5 e 10 anos e como adolescentes, pessoas que estão acima de 10 anos.

In [0]:
# Importar funções do pyspark
from pyspark.sql import functions as F

# Criando a coluna "Faixa_Etaria" baseada na coluna "Age"
df_usuarios = df.withColumn(
    "Faixa_Etaria", 
    F.when((df["Age"] >= 5) & (df["Age"] <= 10), "criança")
     .when((df["Age"] > 10) & (df["Age"] <= 15), "adolescente")
)

# Agrupar por genero, faixa etária, tipo de acesso e mostrar a média de tempo de tela
result = df_usuarios.where(df["Screen_Time_Type"] != "Total").groupBy("Gender", "Faixa_Etaria", "Screen_time_type") \
    .agg(F.avg("Average_Screen_Time_hours").alias("Average_Screen_Time_hours"))

# Mostrar o resultado
result = result.orderBy(F.desc("Average_Screen_Time_hours"))
display(result)

Gender,Faixa_Etaria,Screen_time_type,Average_Screen_Time_hours
Male,adolescente,Recreational,4.703999996185303
Other/Prefer not to say,adolescente,Recreational,4.584999942779541
Female,adolescente,Recreational,4.229999995231628
Male,criança,Recreational,2.2725000182787576
Other/Prefer not to say,criança,Recreational,2.246666669845581
Female,criança,Recreational,2.157500003774961
Female,adolescente,Educational,1.7809999942779542
Male,adolescente,Educational,1.6179999947547912
Other/Prefer not to say,adolescente,Educational,1.565000021457672
Male,criança,Educational,0.900833343466123


Aqui podemos observar que, para o uso recreativo, todos os usuários adolescentes tem mais tempo de tela que os usuários crianças. Também pode-se notar que para o uso recreativo, tanto crianças quanto adolescentes homens ficam mais online que as mulheres.
Porém, para o uso educacional, as adolescentes mulheres usam mais as telas que os homens, quanto que na fase da infância é observado o contrário.

# 5. Criando tabela a partir de dataframe

O dataframe usado para as analises até agora será salvo como tabela para que seja possível conectar ao power bi e mostrar as analises de forma visual em um dashboard.

In [0]:
# Salvar o DataFrame como uma tabela permanente
df.write.saveAsTable("sreen_time_tratado")

In [0]:
%sql
-- Mostrar tabela criada
select * from sreen_time_tratado limit 5


Age,Gender,Screen_Time_Type,Day_Type,Average_Screen_Time_hours,Sample_Size
5,Male,Educational,Weekday,0.44,500
5,Male,Recreational,Weekday,1.11,500
5,Male,Total,Weekday,1.55,500
5,Male,Educational,Weekend,0.5,500
5,Male,Recreational,Weekend,1.44,500


# 6. Visuzalização no Power Bi

A tabela criada foi importada para o Power Bi desktop:
![conectar_powerbi_databricks.png](./conectar_powerbi_databricks.png "conectar_powerbi_databricks.png")
Após ser importada, foram criados os seguintes visuais:![visuais_power_bi_.png](./visuais_power_bi_.png "visuais_power_bi_.png")
Cartões na parte superior indicando a média de tempo para uso receativo, educacional e total;
Gráfico de árvore para visualização de informações de tipo de acesso por usuário (quebra de média de tempo total de telas por tipo de acesso, gênero e faixa etária);
Gráfico de barras clusterizadas e colunas, com tempo de tela por idade e tipos de acesso;
Gráfico de pizza com tempo de tela por gênero;
Filtros para segmentação de dados.

Para que fosse possível criar as visualizações, foi criada uma nova coluna na tabela importada usando a liguagem Dax, para agrupar as idades por faixa etária.
![coluna_calculada.png](./coluna_calculada.png "coluna_calculada.png")

E também foram criadas três medidas para mostrar as informações de tempo de tela por cada tipo de acesso: ![medida_dax.png](./medida_dax.png "medida_dax.png")

# 7. Conclusão
O desenvolvimento do projeto MVP para a disciplina de Engenharia de dados permitiu o aprendizado prático dos conceitos de ETL e tratamento de dados. Além de possibilitar o conhecimento sobre o funcionamento de ferramentas como a Azure, Databricks e Power Bi, e como elas se interagem.

Além disso, foi possível atingir os objetivos do projeto, com a observação da distribuição de tempo de tela entre crianças e jovens para diferentes fins.
Pode-se concluir que o tempo gasto online aumenta com a idade e o principal fator responsável por esse aumento é o uso recreativo. Além disso, quando são considerados fins recreativos, homens adolescentes passam mais tempo em frente a telas que mulheres na mesma faixa etária.
Com isso, é possível debater quais as melhores condutas para que crianças e adolescentes consigam usufruir das tecnologias existentes, mas garantindo o acesso a conteúdos de qualidade.